In [2]:
import os

# Set Spark version
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
!tar xf spark-3.4.3-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.4.3-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Download the file to the Spark context
spark.sparkContext.addFile(url)



In [7]:
# 2. Create a temporary view of the DataFrame.
# Read the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the first few rows to confirm the data was read correctly
df.show(5)

# 2. Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [12]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

# SQL query to calculate the average price for four-bedroom houses per year, formatted to two decimal places
query = """
SELECT
    YEAR(date) AS year_sold,
    FORMAT_NUMBER(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    year_sold
ORDER BY
    year_sold
"""

# Execute the query
avg_price_per_year = spark.sql(query)

# Print a header line
print("Average Price for Four-Bedroom Houses Sold Per Year (Rounded to Two Decimal Places):\n")

# Show the result
avg_price_per_year.show()

Average Price for Four-Bedroom Houses Sold Per Year (Rounded to Two Decimal Places):

+---------+----------+
|year_sold| avg_price|
+---------+----------+
|     2019|300,263.70|
|     2020|298,353.78|
|     2021|301,819.44|
|     2022|296,363.88|
+---------+----------+



In [13]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# SQL query to calculate the average price for homes with 3 bedrooms and 3 bathrooms per year built
query = """
SELECT
    date_built,
    FORMAT_NUMBER(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
avg_price_per_year_built = spark.sql(query)

# Print a header line
print("Average Price of Homes with 3 Bedrooms and 3 Bathrooms Per Year Built (Rounded to Two Decimal Places):\n")

# Show the result
avg_price_per_year_built.show()


Average Price of Homes with 3 Bedrooms and 3 Bathrooms Per Year Built (Rounded to Two Decimal Places):

+----------+----------+
|date_built| avg_price|
+----------+----------+
|      2010|292,859.62|
|      2011|291,117.47|
|      2012|293,683.19|
|      2013|295,962.27|
|      2014|290,852.27|
|      2015|288,770.30|
|      2016|290,555.07|
|      2017|292,676.79|
+----------+----------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# SQL query to calculate the average price for homes with 3 bedrooms, 3 bathrooms, 2 floors,
# and >= 2,000 square feet, per year built
query = """
SELECT
    date_built,
    FORMAT_NUMBER(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
avg_price_per_year_built = spark.sql(query)

# Print a header line
print("Average Price of Homes with 3 Bedrooms, 3 Bathrooms, 2 Floors, and ≥ 2,000 Sqft Per Year Built (Rounded to Two Decimal Places):\n")

# Show the result
avg_price_per_year_built.show()


Average Price of Homes with 3 Bedrooms, 3 Bathrooms, 2 Floors, and ≥ 2,000 Sqft Per Year Built (Rounded to Two Decimal Places):

+----------+----------+
|date_built| avg_price|
+----------+----------+
|      2010|285,010.22|
|      2011|276,553.81|
|      2012|307,539.97|
|      2013|303,676.79|
|      2014|298,264.72|
|      2015|297,609.97|
|      2016|293,965.10|
|      2017|280,317.58|
+----------+----------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Start the timer
start_time = time.time()

# SQL query to calculate the average price per "view" rating
query = """
SELECT
    view,
    FORMAT_NUMBER(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
avg_price_per_view = spark.sql(query)

# Show the result
avg_price_per_view.show()

# Print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|   avg_price|
+----+------------+
| 100|1,026,669.50|
|  99|1,061,201.42|
|  98|1,053,739.33|
|  97|1,129,040.15|
|  96|1,017,815.92|
|  95|1,054,325.60|
|  94|1,033,536.20|
|  93|1,026,006.06|
|  92|  970,402.55|
|  91|1,137,372.73|
|  90|1,062,654.16|
|  89|1,107,839.15|
|  88|1,031,719.35|
|  87|1,072,285.20|
|  86|1,070,444.25|
|  85|1,056,336.74|
|  84|1,117,233.13|
|  83|1,033,965.93|
|  82|1,063,498.00|
|  81|1,053,472.79|
+----+------------+
only showing top 20 rows

--- 1.514868974685669 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached('home_sales')

# Print the result
print(f"Is 'home_sales' cached? {is_cached}")

Is 'home_sales' cached? True


In [21]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

# Start the timer for the cached query
start_time = time.time()

# SQL query to calculate the average price per "view" rating on the cached data
query = """
SELECT
    view,
    FORMAT_NUMBER(AVG(price), 2) AS avg_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query on cached data
avg_price_per_view_cached = spark.sql(query)

# Show the result
avg_price_per_view_cached.show()

# Measure and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|   avg_price|
+----+------------+
| 100|1,026,669.50|
|  99|1,061,201.42|
|  98|1,053,739.33|
|  97|1,129,040.15|
|  96|1,017,815.92|
|  95|1,054,325.60|
|  94|1,033,536.20|
|  93|1,026,006.06|
|  92|  970,402.55|
|  91|1,137,372.73|
|  90|1,062,654.16|
|  89|1,107,839.15|
|  88|1,031,719.35|
|  87|1,072,285.20|
|  86|1,070,444.25|
|  85|1,056,336.74|
|  84|1,117,233.13|
|  83|1,033,965.93|
|  82|1,063,498.00|
|  81|1,053,472.79|
+----+------------+
only showing top 20 rows

--- 1.1126539707183838 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Specify the output path where you want to save the partitioned data
output_path = "/content/home_sales_partitioned"

# Write the DataFrame to Parquet format, partitioned by the "date_built" field
df.write \
  .mode('overwrite') \
  .partitionBy("date_built") \
  .parquet(output_path)

# Print confirmation message
print(f"Data has been written to {output_path}, partitioned by 'date_built'.")

Data has been written to /content/home_sales_partitioned, partitioned by 'date_built'.


In [23]:
# 11. Read the parquet formatted data.

# Specify the path where the Parquet files were saved
parquet_path = "/content/home_sales_partitioned"

# Read the Parquet files into a DataFrame
df_parquet = spark.read.parquet(parquet_path)

# Show the first few rows to verify that the data was read correctly
df_parquet.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [24]:
# 12. Create a temporary table for the parquet data.

# Create a temporary view for the Parquet DataFrame
df_parquet.createOrReplaceTempView("home_sales_parquet")

# Print confirmation
print("Temporary table 'home_sales_parquet' has been created.")

Temporary table 'home_sales_parquet' has been created.


In [25]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

# Start the timer for the Parquet query
start_time = time.time()

# SQL query to calculate the average price per "view" rating on the Parquet data
query = """
SELECT
    view,
    FORMAT_NUMBER(AVG(price), 2) AS avg_price
FROM
    home_sales_parquet
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query on the Parquet data
avg_price_per_view_parquet = spark.sql(query)

# Show the result
avg_price_per_view_parquet.show()

# Measure and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|   avg_price|
+----+------------+
| 100|1,026,669.50|
|  99|1,061,201.42|
|  98|1,053,739.33|
|  97|1,129,040.15|
|  96|1,017,815.92|
|  95|1,054,325.60|
|  94|1,033,536.20|
|  93|1,026,006.06|
|  92|  970,402.55|
|  91|1,137,372.73|
|  90|1,062,654.16|
|  89|1,107,839.15|
|  88|1,031,719.35|
|  87|1,072,285.20|
|  86|1,070,444.25|
|  85|1,056,336.74|
|  84|1,117,233.13|
|  83|1,033,965.93|
|  82|1,063,498.00|
|  81|1,053,472.79|
+----+------------+
only showing top 20 rows

--- 0.8913478851318359 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.

# Uncache the 'home_sales' table
spark.catalog.uncacheTable("home_sales")

# Print confirmation
print("Temporary table 'home_sales' has been uncached.")

Temporary table 'home_sales' has been uncached.


In [27]:
# 15. Check if the home_sales is no longer cached

# Check if the 'home_sales' table is cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False
